In [20]:
# Make sure you are at the root directory of repo
project_root_name = "MechInterpCodeLLMs"

# Find the project root directory
def find_project_root(current_path, project_name):
    while current_path != os.path.dirname(current_path):
        if project_name in os.listdir(current_path):
            return os.path.join(current_path, project_name)
        current_path = os.path.dirname(current_path)
    raise FileNotFoundError(f"Project root directory '{project_name}' not found.")

# Get the path of the current notebook
current_path = os.path.dirname(os.path.abspath(__file__)) if '__file__' in globals() else os.getcwd()
project_root = find_project_root(current_path, project_root_name)

# Set the project root as the current working directory
os.chdir(project_root)

In [23]:
# Feel free to change above code, your current directory should look like this
# README.md  config.py  data/  experiments/  logs/  plots/  transformer_lens/
%ls

README.md  config.py  data/  experiments/  logs/  plots/  transformer_lens/


In [24]:
# This is a local version of transformer_lens, pip uninstall transformer_lens before reach here
import transformer_lens 

In [25]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np
import einops
from fancy_einsum import einsum
import tqdm.notebook as tqdm
import random
import os
import time
from pathlib import Path
import plotly.express as px
from torch.utils.data import DataLoader

from torchtyping import TensorType as TT
from typing import List, Union, Optional, Callable
from typing_extensions import Literal
from functools import partial
import copy
import itertools
import json

from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
import datasets
from IPython.display import HTML, Markdown
from IPython import get_ipython

# Code to automatically update the HookedTransformer code as its edited without restarting the kernel
ipython = get_ipython()
ipython.magic("load_ext autoreload")
ipython.magic("autoreload 2")

/tmp/ipykernel_3085483/3715311821.py:33: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic("load_ext autoreload")
/tmp/ipykernel_3085483/3715311821.py:34: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic("autoreload 2")


In [26]:
# THIS IS A LOCAL (MODIFIED) VERSION OF TRANSFORMER_LENS - UNINSTALL PIP/CONDA VERSION BEFORE USE!
import transformer_lens
import transformer_lens.utils as utils
import transformer_lens.patching as patching
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import (
    HookedTransformer,
    HookedTransformerConfig,
    FactoredMatrix,
    ActivationCache,
)

In [28]:
# HuggingFace bookkeeping so that the home folder doesn't get filled up
# I have these in my .bashrc and .zshrc and they work in terminal but not recognized by python for some reason

# You dont these to run gpt2-small, if you dont want to mess with hf, just comment this

import os
from config import HF_TOKEN, HF_PATH

os.environ["HF_TOKEN"] = HF_TOKEN
os.environ["TRANSFORMERS_CACHE"] = HF_PATH
os.environ["HF_DATASETS_CACHE"] = HF_PATH
os.environ["HF_HOME"] = HF_PATH

In [29]:
# When using multiple GPUs we use GPU 0 as the primary and switch to the next when it is 90% full
num_gpus = torch.cuda.device_count()
device_id = 0
if num_gpus > 0:
    device = "cuda:0"
else:
    device = "cpu"
    
def check_gpu_memory(max_alloc=0.9):
    if not torch.cuda.is_available():
        return
    global device_id, device
    print("Primary device:", device)
    torch.cuda.empty_cache()
    max_alloc = 1 if max_alloc > 1 else max_alloc
    for gpu in range(num_gpus):
        memory_reserved = torch.cuda.memory_reserved(device=gpu)
        memory_allocated = torch.cuda.memory_allocated(device=gpu)
        total_memory = torch.cuda.get_device_properties(gpu).total_memory 
        print(f"GPU {gpu}: {total_memory / (1024**2):.2f} MB  Allocated: {memory_allocated / (1024**2):.2f} MB  Reserved: {memory_reserved / (1024**2):.2f} MB")
                
        # Check if the current GPU is getting too full, and if so we switch the primary device to the next GPU
        if memory_reserved > max_alloc * total_memory:
            if device_id < num_gpus - 1:
                device_id += 1
                device = f"cuda:{device_id}"
                print(f"Switching primary device to {device}")
            else:
                print("Cannot switch primary device, all GPUs are nearly full")

print("Number of GPUs:", num_gpus)
check_gpu_memory()

def timeit(func):
    """Decorator to measure the execution time of a function."""
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f"Function {func.__name__!r} executed in {end_time - start_time:.4f} seconds")
        return result
    return wrapper

Number of GPUs: 1
Primary device: cuda:0
GPU 0: 12207.00 MB  Allocated: 0.00 MB  Reserved: 0.00 MB


In [30]:
# Create transformer
model = HookedTransformer.from_pretrained("gpt2-small", n_devices=num_gpus)

# We need these so that individual attention heads and MLP inputs can be edited
model.set_use_attn_in(True)
model.set_use_hook_mlp_in(True)
model.set_use_attn_result(True) # Documentation says this easily burns through GPU memory

check_gpu_memory()

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Loaded pretrained model gpt2-small into HookedTransformer
Primary device: cuda:0
GPU 0: 12207.00 MB  Allocated: 635.51 MB  Reserved: 690.00 MB


In [31]:
# Some examples to work with - replace with code examples

prompts = ['When J and M went to the shops, J gave the bag to ', 'When J and M went to the shops, M gave the bag to ', 'When T and J went to the park, J gave the ball to ', 'When T and J went to the park, T gave the ball to ', 'When D and S went to the shops, S gave an apple to ', 'When D and S went to the shops, D gave an apple to ', 'After R and A went to the park, A gave a drink to ', 'After R and A went to the park, R gave a drink to ']
answers = [('M', 'J'), ('J', 'M'), ('T', 'J'), ('J', 'T'), ('D', 'S'), ('S', 'D'), ('R', 'A'), ('A', 'R')]

clean_tokens = model.to_tokens(prompts)
# Swap each adjacent pair, with a hacky list comprehension
corrupted_tokens = clean_tokens[
    [(i+1 if i%2==0 else i-1) for i in range(len(clean_tokens)) ]
    ]
print("Clean string 0", model.to_string(clean_tokens[0]))
print("Corrupted string 0", model.to_string(corrupted_tokens[0]))

answer_token_indices = torch.tensor([[model.to_single_token(answers[i][j]) for j in range(2)] for i in range(len(answers))], device=device)
print("Answer token indices", answer_token_indices)
check_gpu_memory()

Clean string 0 <|endoftext|>When J and M went to the shops, J gave the bag to 
Corrupted string 0 <|endoftext|>When J and M went to the shops, M gave the bag to 
Answer token indices tensor([[44, 41],
        [41, 44],
        [51, 41],
        [41, 51],
        [35, 50],
        [50, 35],
        [49, 32],
        [32, 49]], device='cuda:0')
Primary device: cuda:0
GPU 0: 12207.00 MB  Allocated: 635.52 MB  Reserved: 690.00 MB


In [32]:
# Logit difference metric
def get_logit_diff(logits, answer_token_indices=answer_token_indices, device="cpu"):
    if len(logits.shape) == 3:
        # Get final logits only
        logits = logits[:, -1, :]
    logits = logits.to(answer_token_indices.device)
    correct_logits = logits.gather(1, answer_token_indices[:, 0].unsqueeze(1))
    incorrect_logits = logits.gather(1, answer_token_indices[:, 1].unsqueeze(1))
    return (correct_logits - incorrect_logits).mean()

clean_logits, clean_cache = model.run_with_cache(clean_tokens)
corrupted_logits, corrupted_cache = model.run_with_cache(corrupted_tokens)

clean_logit_diff = get_logit_diff(clean_logits, answer_token_indices).item()
print(f"Clean logit diff: {clean_logit_diff:.4f}")

corrupted_logit_diff = get_logit_diff(corrupted_logits, answer_token_indices).item()
print(f"Corrupted logit diff: {corrupted_logit_diff:.4f}")
check_gpu_memory()

Clean logit diff: 0.1306
Corrupted logit diff: -0.1306
Primary device: cuda:0
GPU 0: 12207.00 MB  Allocated: 1916.72 MB  Reserved: 1936.00 MB


In [33]:
# IOI metric - does this mean indirect object identification? Yes

CLEAN_BASELINE = clean_logit_diff
CORRUPTED_BASELINE = corrupted_logit_diff

def ioi_metric(logits, answer_token_indices=answer_token_indices):
    logits = logits.to(device)
    torch.cuda.empty_cache()
    return (get_logit_diff(logits, answer_token_indices) - CORRUPTED_BASELINE) / (
        CLEAN_BASELINE - CORRUPTED_BASELINE
    )

print(f"Clean Baseline is 1: {ioi_metric(clean_logits).item():.4f}")
print(f"Corrupted Baseline is 0: {ioi_metric(corrupted_logits).item():.4f}")
check_gpu_memory()

Clean Baseline is 1: 1.0000
Corrupted Baseline is 0: 0.0000
Primary device: cuda:0
GPU 0: 12207.00 MB  Allocated: 1916.72 MB  Reserved: 1936.00 MB


In [34]:
Metric = Callable[[TT["batch_and_pos_dims", "d_model"]], float]

## Removing prev cache
The memory shot up during run_with_cache, which I dont think we need later on

In [35]:
# Delete tensors
del clean_logits
del corrupted_logits
del clean_logit_diff 
del corrupted_logit_diff
clean_cache = clean_cache.to("cpu")
del corrupted_cache

# Empty CUDA cache
torch.cuda.empty_cache()

# Optionally check memory to confirm
check_gpu_memory()

Primary device: cuda:0
GPU 0: 12207.00 MB  Allocated: 643.64 MB  Reserved: 710.00 MB


# Attribution patching

In [36]:
filter_not_qkv_input = lambda name: "_input" not in name

def get_cache_fwd_and_bwd(model, tokens, metric):
    model.reset_hooks()
    cache = {}

    def forward_cache_hook(act, hook):
        act = act.to(device)
        torch.cuda.empty_cache()
        cache[hook.name] = act

    model.add_hook(filter_not_qkv_input, forward_cache_hook, "fwd")
    grad_cache = {}

    def backward_cache_hook(act, hook):
        act = act.to(device)
        torch.cuda.empty_cache()
        grad_cache[hook.name] = act

    model.add_hook(filter_not_qkv_input, backward_cache_hook, "bwd")
    
    result = model(tokens).to(device)
    torch.cuda.empty_cache()
    value = metric(result)
    value.backward()

    # Reset hooks and clear unused GPU memory
    value = value.item()
    model.reset_hooks()
    torch.cuda.empty_cache()
    
    cache = ActivationCache(cache, model).to(device)
    grad_cache = ActivationCache(grad_cache, model).to(device)
    
    return value,cache, grad_cache

clean_value, clean_cache, clean_grad_cache = get_cache_fwd_and_bwd(model, clean_tokens, ioi_metric)
print("Clean Value:", clean_value)
print("Clean Activations Cached:", len(clean_cache))
print("Clean Gradients Cached:", len(clean_grad_cache))
check_gpu_memory()

clean_cache = clean_cache.to('cpu')
clean_grad_cache = clean_grad_cache.to('cpu')

check_gpu_memory()

corrupted_value, corrupted_cache, corrupted_grad_cache = get_cache_fwd_and_bwd(model, corrupted_tokens, ioi_metric)
print("Corrupted Value:", corrupted_value)
print("Corrupted Activations Cached:", len(corrupted_cache))
print("Corrupted Gradients Cached:", len(corrupted_grad_cache))
check_gpu_memory()

corrupted_cache = corrupted_cache.to('cpu')
corrupted_grad_cache = corrupted_grad_cache.to('cpu')

torch.cuda.empty_cache()
check_gpu_memory()

Clean Value: 1.0
Clean Activations Cached: 244
Clean Gradients Cached: 244
Primary device: cuda:0
GPU 0: 12207.00 MB  Allocated: 1720.93 MB  Reserved: 1836.00 MB
Primary device: cuda:0
GPU 0: 12207.00 MB  Allocated: 1276.02 MB  Reserved: 1740.00 MB
Corrupted Value: 0.0
Corrupted Activations Cached: 244
Corrupted Gradients Cached: 244
Primary device: cuda:0
GPU 0: 12207.00 MB  Allocated: 1727.43 MB  Reserved: 1956.00 MB
Primary device: cuda:0
GPU 0: 12207.00 MB  Allocated: 1276.02 MB  Reserved: 1740.00 MB


In [37]:
HEAD_NAMES = [
    f"L{l}H{h}" for l in range(model.cfg.n_layers) for h in range(model.cfg.n_heads)
]
HEAD_NAMES_SIGNED = [f"{name}{sign}" for name in HEAD_NAMES for sign in ["+", "-"]]
HEAD_NAMES_QKV = [
    f"{name}{act_name}" for name in HEAD_NAMES for act_name in ["Q", "K", "V"]
]
print(HEAD_NAMES[:5])
print(HEAD_NAMES_SIGNED[:5])
print(HEAD_NAMES_QKV[:5])

['L0H0', 'L0H1', 'L0H2', 'L0H3', 'L0H4']
['L0H0+', 'L0H0-', 'L0H1+', 'L0H1-', 'L0H2+']
['L0H0Q', 'L0H0K', 'L0H0V', 'L0H1Q', 'L0H1K']


In [38]:
@timeit
def create_attention_attr(
    clean_cache, clean_grad_cache, device
) -> TT["batch", "layer", "head_index", "dest", "src"]:
    attention_stack = torch.stack(
        [clean_cache["pattern", l] for l in range(model.cfg.n_layers)], dim=0
    ).to(device)
    attention_grad_stack = torch.stack(
        [clean_grad_cache["pattern", l] for l in range(model.cfg.n_layers)], dim=0
    ).to(device)
    attention_attr = attention_grad_stack * attention_stack
    attention_attr = einops.rearrange(
        attention_attr,
        "layer batch head_index dest src -> batch layer head_index dest src",
    )
    return attention_attr

attention_attr = create_attention_attr(clean_cache, clean_grad_cache, "cpu")
check_gpu_memory()

Function 'create_attention_attr' executed in 0.2138 seconds
Primary device: cuda:0
GPU 0: 12207.00 MB  Allocated: 1276.02 MB  Reserved: 1740.00 MB


In [39]:
torch.cuda.empty_cache()
check_gpu_memory()

Primary device: cuda:0
GPU 0: 12207.00 MB  Allocated: 1276.02 MB  Reserved: 1740.00 MB


In [40]:
# TODO : viz corresponding to plot_attention_attr

# Residual Stream Patching

In [41]:
@timeit
def attr_patch_residual(
    clean_cache: ActivationCache,
    corrupted_cache: ActivationCache,
    corrupted_grad_cache: ActivationCache,
    device,
) -> TT["component", "pos"]:
    clean_residual, residual_labels = clean_cache.accumulated_resid(
        -1, incl_mid=True, return_labels=True
    )
    corrupted_residual = corrupted_cache.accumulated_resid(
        -1, incl_mid=True, return_labels=False
    )
    corrupted_grad_residual = corrupted_grad_cache.accumulated_resid(
        -1, incl_mid=True, return_labels=False
    )
    residual_attr = einops.reduce(
        corrupted_grad_residual * (clean_residual - corrupted_residual),
        "component batch pos d_model -> component pos",
        "sum",
    ).to(device)
    return residual_attr, residual_labels

In [42]:
residual_attr, residual_labels = attr_patch_residual(
    clean_cache, corrupted_cache, corrupted_grad_cache, "cpu"
)
check_gpu_memory()

Function 'attr_patch_residual' executed in 0.0241 seconds
Primary device: cuda:0
GPU 0: 12207.00 MB  Allocated: 1276.02 MB  Reserved: 1740.00 MB


In [ ]:
# TODO : viz corresponding to attr_patch_residual

# Layer Output Attribution Patching

In [43]:
@timeit
def attr_patch_layer_out(
    clean_cache: ActivationCache,
    corrupted_cache: ActivationCache,
    corrupted_grad_cache: ActivationCache,
    device
) -> TT["component", "pos"]:
    clean_layer_out, labels = clean_cache.decompose_resid(-1, return_labels=True)
    corrupted_layer_out = corrupted_cache.decompose_resid(-1, return_labels=False)
    corrupted_grad_layer_out = corrupted_grad_cache.decompose_resid(
        -1, return_labels=False
    )
    layer_out_attr = einops.reduce(
        corrupted_grad_layer_out * (clean_layer_out - corrupted_layer_out),
        "component batch pos d_model -> component pos",
        "sum",
    ).to(device)
    return layer_out_attr, labels

In [44]:
layer_out_attr, layer_out_labels = attr_patch_layer_out(
    clean_cache, corrupted_cache, corrupted_grad_cache, "cpu"
)
check_gpu_memory()

Function 'attr_patch_layer_out' executed in 0.0250 seconds
Primary device: cuda:0
GPU 0: 12207.00 MB  Allocated: 1276.02 MB  Reserved: 1740.00 MB


In [ ]:
# TODO : viz corresponding to attr_patch_layer_out

# Head output attribution Patching

In [45]:
@timeit
def attr_patch_head_out(
    clean_cache: ActivationCache,
    corrupted_cache: ActivationCache,
    corrupted_grad_cache: ActivationCache,
    device
) -> TT["component", "pos"]:
    labels = HEAD_NAMES

    clean_head_out = clean_cache.stack_head_results(-1, return_labels=False).to(device)
    corrupted_head_out = corrupted_cache.stack_head_results(-1, return_labels=False).to(device)
    corrupted_grad_head_out = corrupted_grad_cache.stack_head_results(
        -1, return_labels=False
    ).to(device)
    head_out_attr = einops.reduce(
        corrupted_grad_head_out * (clean_head_out - corrupted_head_out),
        "component batch pos d_model -> component pos",
        "sum",
    ).to(device)
    return head_out_attr, labels

In [46]:
head_out_attr, head_out_labels = attr_patch_head_out(
    clean_cache, corrupted_cache, corrupted_grad_cache, "cpu"
)

sum_head_out_attr = einops.reduce(
    head_out_attr,
    "(layer head) pos -> layer head",
    "sum",
    layer=model.cfg.n_layers,
    head=model.cfg.n_heads,
)
check_gpu_memory()

Function 'attr_patch_head_out' executed in 0.1252 seconds
Primary device: cuda:0
GPU 0: 12207.00 MB  Allocated: 1276.02 MB  Reserved: 1740.00 MB


In [ ]:
# TODO : viz corresponding to attr_patch_head_out

# Head activation patching, skipping it for now

In [47]:
def stack_head_vector_from_cache(
    cache, activation_name: Literal["q", "k", "v", "z"], device
) -> TT["layer_and_head_index", "batch", "pos", "d_head"]:
    """Stacks the head vectors from the cache from a specific activation (key, query, value or mixed_value (z)) into a single tensor."""
    stacked_head_vectors = torch.stack(
        [cache[activation_name, l] for l in range(model.cfg.n_layers)], dim=0
    ).to(device)
    stacked_head_vectors = einops.rearrange(
        stacked_head_vectors,
        "layer batch pos head_index d_head -> (layer head_index) batch pos d_head",
    ).to(device)
    return stacked_head_vectors

@timeit
def attr_patch_head_vector(
    clean_cache: ActivationCache,
    corrupted_cache: ActivationCache,
    corrupted_grad_cache: ActivationCache,
    activation_name: Literal["q", "k", "v", "z"],
    device
) -> TT["component", "pos"]:
    labels = HEAD_NAMES

    clean_head_vector = stack_head_vector_from_cache(clean_cache, activation_name, "cpu").to(device)
    corrupted_head_vector = stack_head_vector_from_cache(
        corrupted_cache, activation_name, "cpu"
    ).to(device)
    corrupted_grad_head_vector = stack_head_vector_from_cache(
        corrupted_grad_cache, activation_name, "cpu"
    ).to(device)
    head_vector_attr = einops.reduce(
        corrupted_grad_head_vector * (clean_head_vector - corrupted_head_vector),
        "component batch pos d_head -> component pos",
        "sum",
    )
    return head_vector_attr, labels

In [48]:
head_vector_attr_dict = {}
for activation_name, activation_name_full in [
    ("k", "Key"),
    ("q", "Query"),
    ("v", "Value"),
    ("z", "Mixed Value"),
]:
    head_vector_attr_dict[activation_name], head_vector_labels = attr_patch_head_vector(
        clean_cache, corrupted_cache, corrupted_grad_cache, activation_name, "cpu"
    )
    sum_head_vector_attr = einops.reduce(
        head_vector_attr_dict[activation_name],
        "(layer head) pos -> layer head",
        "sum",
        layer=model.cfg.n_layers,
        head=model.cfg.n_heads,
    )
check_gpu_memory()

Function 'attr_patch_head_vector' executed in 0.0160 seconds
Function 'attr_patch_head_vector' executed in 0.0051 seconds
Function 'attr_patch_head_vector' executed in 0.0053 seconds
Function 'attr_patch_head_vector' executed in 0.0088 seconds
Primary device: cuda:0
GPU 0: 12207.00 MB  Allocated: 1276.02 MB  Reserved: 1740.00 MB


In [ ]:
# TODO : viz corresponding to attr_patch_head_vector

# Head Pattern Attribution Patching

In [49]:
def stack_head_pattern_from_cache(
    cache,
    device
) -> TT["layer_and_head_index", "batch", "dest_pos", "src_pos"]:
    """Stacks the head patterns from the cache into a single tensor."""
    stacked_head_pattern = torch.stack(
        [cache["pattern", l] for l in range(model.cfg.n_layers)], dim=0
    ).to(device)
    stacked_head_pattern = einops.rearrange(
        stacked_head_pattern,
        "layer batch head_index dest_pos src_pos -> (layer head_index) batch dest_pos src_pos",
    ).to(device)
    return stacked_head_pattern

@timeit
def attr_patch_head_pattern(
    clean_cache: ActivationCache,
    corrupted_cache: ActivationCache,
    corrupted_grad_cache: ActivationCache,
    device
) -> TT["component", "dest_pos", "src_pos"]:
    labels = HEAD_NAMES

    clean_head_pattern = stack_head_pattern_from_cache(clean_cache, "cpu").to(device)
    corrupted_head_pattern = stack_head_pattern_from_cache(corrupted_cache, "cpu").to(device)
    corrupted_grad_head_pattern = stack_head_pattern_from_cache(corrupted_grad_cache, "cpu").to(device)
    head_pattern_attr = einops.reduce(
        corrupted_grad_head_pattern * (clean_head_pattern - corrupted_head_pattern),
        "component batch dest_pos src_pos -> component dest_pos src_pos",
        "sum",
    ).to(device)
    return head_pattern_attr, labels

In [50]:
head_pattern_attr, labels = attr_patch_head_pattern(
    clean_cache, corrupted_cache, corrupted_grad_cache, "cpu"
)

head_pattern_attr = einops.rearrange(
        head_pattern_attr,
        "(layer head) dest src -> layer head dest src",
        layer=model.cfg.n_layers,
        head=model.cfg.n_heads,
    )
check_gpu_memory()

Function 'attr_patch_head_pattern' executed in 0.0028 seconds
Primary device: cuda:0
GPU 0: 12207.00 MB  Allocated: 1276.02 MB  Reserved: 1740.00 MB


In [51]:
# TODO : viz corresponding to attr_patch_head_pattern

# Head Path Attribution Patching

In [52]:
def get_head_vector_grad_input_from_grad_cache(
    grad_cache: ActivationCache, activation_name: Literal["q", "k", "v"], layer: int, device
) -> TT["batch", "pos", "head_index", "d_model"]:
    vector_grad = grad_cache[activation_name, layer].to(device)
    ln_scales = grad_cache["scale", layer, "ln1"].to(device)
    attn_layer_object = model.blocks[layer].attn
    if activation_name == "q":
        W = attn_layer_object.W_Q.to(device)
    elif activation_name == "k":
        W = attn_layer_object.W_K.to(device)
    elif activation_name == "v":
        W = attn_layer_object.W_V.to(device)
    else:
        raise ValueError("Invalid activation name")

    # Original notebook used (batch pos) for second input but that seems to be wrong - double check this computation
    return einsum(
        "batch pos head_index d_head, batch pos head_index, head_index d_model d_head -> batch pos head_index d_model",
        vector_grad,
        ln_scales.squeeze(-1),
        W,
    )

def get_stacked_head_vector_grad_input(
    grad_cache, activation_name: Literal["q", "k", "v"], device
) -> TT["layer", "batch", "pos", "head_index", "d_model"]:
    return torch.stack(
        [
            get_head_vector_grad_input_from_grad_cache(grad_cache, activation_name, l, "cpu")
            for l in range(model.cfg.n_layers)
        ],
        dim=0,
    ).to(device)

def get_full_vector_grad_input(
    grad_cache, device
) -> TT["qkv", "layer", "batch", "pos", "head_index", "d_model"]:
    return torch.stack([get_stacked_head_vector_grad_input(grad_cache, activation_name, "cpu").to(device) for activation_name in ["q", "k", "v"]], dim=0).to(device)

@timeit
def attr_patch_head_path(
    clean_cache: ActivationCache,
    corrupted_cache: ActivationCache,
    corrupted_grad_cache: ActivationCache,
    device
) -> TT["qkv", "dest_component", "src_component", "pos"]:
    """
    Computes the attribution patch along the path between each pair of heads.

    Sets this to zero for the path from any late head to any early head

    """
    start_labels = HEAD_NAMES
    end_labels = HEAD_NAMES_QKV
    full_vector_grad_input = get_full_vector_grad_input(corrupted_grad_cache, "cpu")
    clean_head_result_stack = clean_cache.stack_head_results(-1)
    corrupted_head_result_stack = corrupted_cache.stack_head_results(-1)
    diff_head_result = einops.rearrange(
        clean_head_result_stack - corrupted_head_result_stack,
        "(layer head_index) batch pos d_model -> layer batch pos head_index d_model",
        layer=model.cfg.n_layers,
        head_index=model.cfg.n_heads,
    )
    path_attr = einsum(
        "qkv layer_end batch pos head_end d_model, layer_start batch pos head_start d_model -> qkv layer_end head_end layer_start head_start pos",
        full_vector_grad_input,
        diff_head_result,
    )
    correct_layer_order_mask = (
        torch.arange(model.cfg.n_layers)[None, :, None, None, None, None]
        > torch.arange(model.cfg.n_layers)[None, None, None, :, None, None]
    ).to(path_attr.device)
    zero = torch.zeros(1, device=path_attr.device)
    path_attr = torch.where(correct_layer_order_mask, path_attr, zero)

    path_attr = einops.rearrange(
        path_attr,
        "qkv layer_end head_end layer_start head_start pos -> (layer_end head_end qkv) (layer_start head_start) pos",
    )
    return path_attr, end_labels, start_labels


In [53]:
head_path_attr, end_labels, start_labels = attr_patch_head_path(
    clean_cache, corrupted_cache, corrupted_grad_cache, "cpu"
)

Function 'attr_patch_head_path' executed in 0.5152 seconds


In [54]:
check_gpu_memory()

Primary device: cuda:0
GPU 0: 12207.00 MB  Allocated: 1276.02 MB  Reserved: 1740.00 MB


In [55]:
# TODO : viz corresponding to attr_patch_head_path

In [56]:
head_out_values, head_out_indices = head_out_attr.sum(-1).abs().sort(descending=True)
top_head_indices = head_out_indices[:22].sort().values
top_end_indices = []
top_end_labels = []
top_start_indices = []
top_start_labels = []

for i in top_head_indices:
    i = i.item()
    top_start_indices.append(i)
    top_start_labels.append(start_labels[i])
    for j in range(3):
        top_end_indices.append(3 * i + j)
        top_end_labels.append(end_labels[3 * i + j])
        
top_head_path_attr = einops.rearrange(
    head_path_attr[top_end_indices, :][:, top_start_indices].sum(-1),
    "(head_end qkv) head_start -> qkv head_end head_start",
    qkv=3,
)

check_gpu_memory()

# TODO : viz corresponding to attr_patch_head_path

Primary device: cuda:0
GPU 0: 12207.00 MB  Allocated: 1276.02 MB  Reserved: 1740.00 MB


In [57]:
# TODO : viz corresponding to attr_patch_head_path
# for j, composition_type in enumerate(["Query", "Key", "Value"]):
#     imshow(
#         head_path_attr[top_end_indices, :][:, top_start_indices][j::3].sum(-1),
#         y=top_end_labels[j::3],
#         yaxis="Path End (Head Input)",
#         x=top_start_labels,
#         xaxis="Path Start (Head Output)",
#         title=f"Head Path to {composition_type} Attribution Patching (Filtered for Top Heads)",
#     )

In [58]:
top_head_path_attr = einops.rearrange(
    head_path_attr[top_end_indices, :][:, top_start_indices].sum(-1),
    "(head_end qkv) head_start -> qkv head_end head_start",
    qkv=3,
)
# TODO : viz corresponding to attr_patch_head_path
# imshow(
#     top_head_path_attr,
#     y=[i[:-1] for i in top_end_labels[::3]],
#     yaxis="Path End (Head Input)",
#     x=top_start_labels,
#     xaxis="Path Start (Head Output)",
#     title=f"Head Path Attribution Patching (Filtered for Top Heads)",
#     facet_col=0,
#     facet_labels=["Query", "Key", "Value"],
# )

In [59]:
interesting_heads = [
    5 * model.cfg.n_heads + 5,
    8 * model.cfg.n_heads + 6,
    9 * model.cfg.n_heads + 9,
]
interesting_head_labels = [HEAD_NAMES[i] for i in interesting_heads]
for head_index, label in zip(interesting_heads, interesting_head_labels):
    in_paths = head_path_attr[3 * head_index : 3 * head_index + 3].sum(-1)
    out_paths = head_path_attr[:, head_index].sum(-1)
    out_paths = einops.rearrange(out_paths, "(layer_head qkv) -> qkv layer_head", qkv=3)
    all_paths = torch.cat([in_paths, out_paths], dim=0)
    all_paths = einops.rearrange(
        all_paths,
        "path_type (layer head) -> path_type layer head",
        layer=model.cfg.n_layers,
        head=model.cfg.n_heads,
    )
# TODO - implement visualization for input/output paths per head
    
check_gpu_memory()

Primary device: cuda:0
GPU 0: 12207.00 MB  Allocated: 1276.02 MB  Reserved: 1740.00 MB


# Validating Attribution vs Activation Patching

In [60]:
attribution_cache_dict = {}
for key in corrupted_grad_cache.cache_dict.keys():
    attribution_cache_dict[key] = corrupted_grad_cache.cache_dict[key] * (
        clean_cache.cache_dict[key] - corrupted_cache.cache_dict[key]
    ).to("cpu")
attr_cache = ActivationCache(attribution_cache_dict, model).to("cpu")
check_gpu_memory()

Primary device: cuda:0
GPU 0: 12207.00 MB  Allocated: 1276.02 MB  Reserved: 1740.00 MB


# Activation Patching per Block, skipping it for now

In [16]:
device

'cuda:0'

In [61]:
str_tokens = model.to_str_tokens(clean_tokens[0])
context_length = len(str_tokens)
every_block_act_patch_result = patching.get_act_patch_block_every(
    model, corrupted_tokens, clean_cache, ioi_metric
).to("cpu")


# TODO : viz corresponding to Activation Patching per block
check_gpu_memory()

  0%|          | 0/192 [00:00<?, ?it/s]

  0%|          | 0/192 [00:00<?, ?it/s]

  0%|          | 0/192 [00:00<?, ?it/s]

Primary device: cuda:0
GPU 0: 12207.00 MB  Allocated: 1276.02 MB  Reserved: 1740.00 MB


# Attribution Patching per Block

In [63]:
@timeit
def get_attr_patch_block_every(attr_cache, device):
    resid_pre_attr = einops.reduce(
        attr_cache.stack_activation("resid_pre"),
        "layer batch pos d_model -> layer pos",
        "sum",
    ).to(device)
    attn_out_attr = einops.reduce(
        attr_cache.stack_activation("attn_out"),
        "layer batch pos d_model -> layer pos",
        "sum",
    ).to(device)
    mlp_out_attr = einops.reduce(
        attr_cache.stack_activation("mlp_out"),
        "layer batch pos d_model -> layer pos",
        "sum",
    ).to(device)

    every_block_attr_patch_result = torch.stack(
        [resid_pre_attr, attn_out_attr, mlp_out_attr], dim=0
    )
    return every_block_attr_patch_result

In [64]:
every_block_attr_patch_result = get_attr_patch_block_every(attr_cache, "cpu")
check_gpu_memory()
# TODO : viz corresponding to Attribution Patching per block

Function 'get_attr_patch_block_every' executed in 0.0121 seconds
Primary device: cuda:0
GPU 0: 12207.00 MB  Allocated: 1276.02 MB  Reserved: 1740.00 MB


In [65]:
# TODO : viz corresponding to Activation vs Attribution Patching per block

# Activation Patching per Head

In [66]:
every_head_all_pos_act_patch_result = patching.get_act_patch_attn_head_all_pos_every(
    model, corrupted_tokens, clean_cache, ioi_metric
)

# # TODO : viz corresponding to Activation Patching per head

  0%|          | 0/144 [00:00<?, ?it/s]

  0%|          | 0/144 [00:00<?, ?it/s]

  0%|          | 0/144 [00:00<?, ?it/s]

  0%|          | 0/144 [00:00<?, ?it/s]

  0%|          | 0/144 [00:00<?, ?it/s]

# Attribution Patching per Head

In [67]:
@timeit
def get_attr_patch_attn_head_all_pos_every(attr_cache, device):
    head_out_all_pos_attr = einops.reduce(
        attr_cache.stack_activation("z"),
        "layer batch pos head_index d_head -> layer head_index",
        "sum",
    )
    head_q_all_pos_attr = einops.reduce(
        attr_cache.stack_activation("q"),
        "layer batch pos head_index d_head -> layer head_index",
        "sum",
    )
    head_k_all_pos_attr = einops.reduce(
        attr_cache.stack_activation("k"),
        "layer batch pos head_index d_head -> layer head_index",
        "sum",
    )
    head_v_all_pos_attr = einops.reduce(
        attr_cache.stack_activation("v"),
        "layer batch pos head_index d_head -> layer head_index",
        "sum",
    )
    head_pattern_all_pos_attr = einops.reduce(
        attr_cache.stack_activation("pattern"),
        "layer batch head_index dest_pos src_pos -> layer head_index",
        "sum",
    )

    return torch.stack(
        [
            head_out_all_pos_attr,
            head_q_all_pos_attr,
            head_k_all_pos_attr,
            head_v_all_pos_attr,
            head_pattern_all_pos_attr,
        ]
    )


every_head_all_pos_attr_patch_result = get_attr_patch_attn_head_all_pos_every(
    attr_cache, "cpu"
)

# TODO : viz corresponding to Attribution Patching per block

Function 'get_attr_patch_attn_head_all_pos_every' executed in 0.0183 seconds


In [68]:
# TODO : viz corresponding to Activation vs Attribution Patching per block

In [69]:
# TODO : viz corresponding to Attention for specific heads
graph_tok_labels = [
    f"{tok}_{i}" for i, tok in enumerate(model.to_str_tokens(clean_tokens[0]))
]
# imshow(
#     clean_cache["pattern", 5][:, 5],
#     x=graph_tok_labels,
#     y=graph_tok_labels,
#     facet_col=0,
#     title="Attention for Head L5H5",
#     facet_name="Prompt",
# )
# imshow(
#     clean_cache["pattern", 10][:, 7],
#     x=graph_tok_labels,
#     y=graph_tok_labels,
#     facet_col=0,
#     title="Attention for Head L10H7",
#     facet_name="Prompt",
# )
# imshow(
#     clean_cache["pattern", 11][:, 10],
#     x=graph_tok_labels,
#     y=graph_tok_labels,
#     facet_col=0,
#     title="Attention for Head L11H10",
#     facet_name="Prompt",
# )


In [ ]:
every_head_by_pos_act_patch_result = patching.get_act_patch_attn_head_by_pos_every(
    model, corrupted_tokens, clean_cache, ioi_metric
)
every_head_by_pos_act_patch_result = einops.rearrange(
    every_head_by_pos_act_patch_result,
    "act_type layer pos head -> act_type (layer head) pos",
)
# TODO : viz corresponding to Attention per head

In [ ]:
# TODO : viz corresponding to Attention for specific heads again
graph_tok_labels = [
    f"{tok}_{i}" for i, tok in enumerate(model.to_str_tokens(clean_tokens[0]))
]
# imshow(
#     clean_cache["pattern", 5][:, 5],
#     x=graph_tok_labels,
#     y=graph_tok_labels,
#     facet_col=0,
#     title="Attention for Head L5H5",
#     facet_name="Prompt",
# )
# imshow(
#     clean_cache["pattern", 10][:, 7],
#     x=graph_tok_labels,
#     y=graph_tok_labels,
#     facet_col=0,
#     title="Attention for Head L10H7",
#     facet_name="Prompt",
# )
# imshow(
#     clean_cache["pattern", 11][:, 10],
#     x=graph_tok_labels,
#     y=graph_tok_labels,
#     facet_col=0,
#     title="Attention for Head L11H10",
#     facet_name="Prompt",
# )


# [markdown]

In [ ]:
# TODO : viz corresponding to Attention patching per head (by pos)

every_head_by_pos_act_patch_result = patching.get_act_patch_attn_head_by_pos_every(
    model, corrupted_tokens, clean_cache, ioi_metric
)
every_head_by_pos_act_patch_result = einops.rearrange(
    every_head_by_pos_act_patch_result,
    "act_type layer pos head -> act_type (layer head) pos",
)
imshow(
    every_head_by_pos_act_patch_result,
    facet_col=0,
    facet_labels=["Output", "Query", "Key", "Value", "Pattern"],
    title="Activation Patching Per Head (By Pos)",
    xaxis="Position",
    yaxis="Layer & Head",
    zmax=1,
    zmin=-1,
    x=[f"{tok}_{i}" for i, tok in enumerate(model.to_str_tokens(clean_tokens[0]))],
    y=head_out_labels,
)

In [ ]:
def get_attr_patch_attn_head_by_pos_every(attr_cache):
    head_out_by_pos_attr = einops.reduce(
        attr_cache.stack_activation("z"),
        "layer batch pos head_index d_head -> layer pos head_index",
        "sum",
    )
    head_q_by_pos_attr = einops.reduce(
        attr_cache.stack_activation("q"),
        "layer batch pos head_index d_head -> layer pos head_index",
        "sum",
    )
    head_k_by_pos_attr = einops.reduce(
        attr_cache.stack_activation("k"),
        "layer batch pos head_index d_head -> layer pos head_index",
        "sum",
    )
    head_v_by_pos_attr = einops.reduce(
        attr_cache.stack_activation("v"),
        "layer batch pos head_index d_head -> layer pos head_index",
        "sum",
    )
    head_pattern_by_pos_attr = einops.reduce(
        attr_cache.stack_activation("pattern"),
        "layer batch head_index dest_pos src_pos -> layer dest_pos head_index",
        "sum",
    )

    return torch.stack(
        [
            head_out_by_pos_attr,
            head_q_by_pos_attr,
            head_k_by_pos_attr,
            head_v_by_pos_attr,
            head_pattern_by_pos_attr,
        ]
    )


every_head_by_pos_attr_patch_result = get_attr_patch_attn_head_by_pos_every(attr_cache)
every_head_by_pos_attr_patch_result = einops.rearrange(
    every_head_by_pos_attr_patch_result,
    "act_type layer pos head -> act_type (layer head) pos",
)
# TODO : viz corresponding to attribution patching per head (by pos)
# imshow(
#     every_head_by_pos_attr_patch_result,
#     facet_col=0,
#     facet_labels=["Output", "Query", "Key", "Value", "Pattern"],
#     title="Attribution Patching Per Head (By Pos)",
#     xaxis="Position",
#     yaxis="Layer & Head",
#     zmax=1,
#     zmin=-1,
#     x=[f"{tok}_{i}" for i, tok in enumerate(model.to_str_tokens(clean_tokens[0]))],
#     y=head_out_labels,
# )

In [ ]:
# TODO : viz corresponding to attribution vs Activation patching per head (by pos)